In [2]:
import pandas as pd
pd.options.display.max_columns = 200
pd.options.display.float_format = '{:.3f}'.format

/Users/margaryta/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
import warnings
warnings.filterwarnings('ignore')

[# Read links/Users/margaryta/Desktop/WILD/Project 2/tmdb_full.csv

In [5]:
# Read the right files to create our DB

link_bas = r"/Users/margaryta/Desktop/WILD/Project_2/title.basics.tsv.gz"
title_bas = pd.read_csv(link_bas, sep = "\t", low_memory = False, na_values = r"\N")


In [4]:
link_akas = r"/Users/margaryta/Desktop/WILD/Project 2/title.akas.tsv.gz"
title_akas = pd.read_csv(link_akas, sep = "\t", low_memory = False, na_values = r"\N")

In [5]:
title_bas[title_bas["titleType"] == "movie"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 677099 entries, 8 to 10687954
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          677099 non-null  object 
 1   titleType       677099 non-null  object 
 2   primaryTitle    677099 non-null  object 
 3   originalTitle   677099 non-null  object 
 4   isAdult         677099 non-null  float64
 5   startYear       579191 non-null  float64
 6   endYear         0 non-null       float64
 7   runtimeMinutes  425980 non-null  object 
 8   genres          602386 non-null  object 
dtypes: float64(3), object(6)
memory usage: 51.7+ MB


In [6]:
link_rate = r"/Users/margaryta/Desktop/WILD/Project 2/title.ratings.tsv.gz"
title_rate = pd.read_csv(link_rate, sep = "\t", na_values = r"\N")

In [7]:
TMdb = pd.read_csv(r"/Users/margaryta/Desktop/WILD/Project 2/tmdb_full.csv", low_memory = False)

In [8]:
link_prin = r"/Users/margaryta/Desktop/WILD/Project 2/title.principals.tsv.gz"
title_prin = pd.read_csv(link_prin, sep = "\t", na_values = r"\N")

In [9]:
link_name = r"/Users/margaryta/Desktop/WILD/Project 2/name.basics.tsv.gz"
name_bas = pd.read_csv(link_name, sep = "\t", na_values = r"\N", low_memory = False)

# Run from here after reading the links

In [10]:
# Get the list of existing type in title_basics
title_bas["titleType"].unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [11]:
# Creation of the DB with only movies without genre Adult
df_movie = title_bas[(~title_bas["genres"].str.contains("Adult", na = False)) & (title_bas["titleType"].str.contains("movie", na = False))]

In [13]:
# Creation of the list of traduced title
fr_akas = title_akas[["titleId","title", "types", "region"]][title_akas["region"].str.contains("FR", na = False)]

In [15]:
# Add the title type on akas to filter just the movies
fr_akas = fr_akas.merge(title_bas["titleType"], how = "left", left_on = fr_akas["titleId"], right_on = title_bas["tconst"])

In [16]:
fr_akas = fr_akas[fr_akas["titleType"] == "movie"]

In [18]:
fr_akas[fr_akas.duplicated(subset = "titleId", keep = False)]

,key_0,titleId,title,types,region,titleType
532,tt0002423,tt0002423,La Du Barry,NaN,FR,movie
533,tt0002423,tt0002423,Madame du Barry,alternative,FR,movie
534,tt0002423,tt0002423,Passion,imdbDisplay,FR,movie
566,tt0002844,tt0002844,The Phantom Crook,imdbDisplay,FR,movie
567,tt0002844,tt0002844,Fantômas,NaN,FR,movie
...,...,...,...,...,...,...
4690671,tt9896916,tt9896916,Le Voyage du Pèlerin,imdbDisplay,FR,movie
4692193,tt9904530,tt9904530,Scream Returns,imdbDisplay,FR,movie
4692194,tt9904530,tt9904530,Scream Returns - Fan Film Spin-Off,NaN,FR,movie
4693018,tt9907782,tt9907782,The Cursed,alternative,FR,movie


In [19]:
dupli_akas = fr_akas[fr_akas.duplicated(subset = "titleId", keep = False)]

In [20]:
len(dupli_akas["titleId"].unique())

8183

In [21]:
len(dupli_akas["titleId"][dupli_akas["types"] == "imdbDisplay"].unique())

6603

In [23]:
dupli_akas["types"] = dupli_akas["types"].apply(lambda x: "aaimdbDisplay" if x == "imdbDisplay" else x)

In [24]:
dupli_akas.sort_values(by = "types", inplace = True)


In [25]:
# Removing the duplicated lines (same movie with different fr akas)
dupli_akas.drop_duplicates(subset = ["titleId"], inplace = True)

In [26]:
# Removing all duplicated lines (same movie with different fr akas)
fr_akas.drop_duplicates(subset = ["titleId"], keep = False, inplace = True)

In [27]:
# Add the unique title from the duplicated list 
fr_akas = pd.concat([fr_akas, dupli_akas])

In [29]:
fr_akas["titleId"].value_counts()

tt0000591     1
tt19354890    1
tt1935302     1
tt19352286    1
tt19352108    1
             ..
tt0135064     1
tt0135049     1
tt0135037     1
tt0135033     1
tt9234412     1
Name: titleId, Length: 85244, dtype: int64

In [30]:
# Merging (INNER) movies and akas to get only the translated movies
movies_frt = df_movie.merge(fr_akas["title"], how = "inner", left_on = df_movie["tconst"], right_on = fr_akas["titleId"])

In [33]:
movies_frt.drop(columns = "key_0", inplace = True)

In [34]:
# Merging (LEFT) fr_movies with ratings
movies_frt = movies_frt.merge(title_rate[["averageRating", "numVotes"]], how = "left", left_on = movies_frt["tconst"], right_on = title_rate["tconst"])

In [35]:
movies_frt.drop(columns = "key_0", inplace = True)

In [36]:
# Final selection of movies before dropping columns

movie_final = movies_frt.merge(TMdb, how = "left", left_on = movies_frt["tconst"], right_on = TMdb["imdb_id"])

In [37]:
display(movie_final.info(),
        movie_final.describe(),
        movie_final.sample(10))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83878 entries, 0 to 83877
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   key_0                         83878 non-null  object 
 1   tconst                        83878 non-null  object 
 2   titleType                     83878 non-null  object 
 3   primaryTitle                  83878 non-null  object 
 4   originalTitle                 83878 non-null  object 
 5   isAdult                       83878 non-null  float64
 6   startYear                     82314 non-null  float64
 7   endYear                       0 non-null      float64
 8   runtimeMinutes                77237 non-null  object 
 9   genres_x                      80218 non-null  object 
 10  title_x                       83878 non-null  object 
 11  averageRating                 67760 non-null  float64
 12  numVotes                      67760 non-null  float64
 13  a

None

,isAdult,startYear,endYear,averageRating,numVotes,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,83878.000,82314.000,0.000,67760.000,67760.000,60617.000,60617.000,60617.000,60617.000,60617.000,60617.000,60617.000
mean,0.002,1987.454,NaN,6.112,15432.330,4106393.290,256153.376,7.450,10971796.174,94.789,5.416,299.406
std,0.044,31.049,NaN,1.117,75769.678,17983797.856,275890.333,46.967,66945949.151,184.540,2.082,1300.024
min,0.000,1903.000,NaN,1.000,5.000,0.000,2.000,0.600,0.000,0.000,0.000,0.000
25%,0.000,1965.000,NaN,5.500,113.000,0.000,43499.000,1.220,0.000,86.000,5.000,4.000
50%,0.000,1997.000,NaN,6.200,624.000,0.000,135829.000,2.687,0.000,95.000,6.000,16.000
75%,0.000,2015.000,NaN,6.900,3654.000,0.000,407887.000,7.407,0.000,105.000,6.700,89.000
max,1.000,2031.000,NaN,10.000,2881263.000,579330426.000,1122335.000,5089.969,2923706026.000,43200.000,10.000,33630.000


,key_0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_x,title_x,averageRating,numVotes,adult,backdrop_path,budget,genres_y,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_y,video,vote_average,vote_count,production_companies_name,production_companies_country
67343,tt2627822,tt2627822,movie,Final Cut,Final Cut,0.000,1989.000,NaN,90,"Action,Crime,Thriller",La scène du crime,5.000,10.000,False,NaN,0.000,"['Action', 'Crime', 'Thriller']",NaN,231561.000,tt2627822,en,Final Cut,‘Snuff film’ with a little boy witnessing a re...,0.600,NaN,['ZA'],1989-10-31,0.000,90.000,['en'],Released,Lights! Cameras! Murder!,Final Cut,False,0.000,0.000,"['Compas Films (PTV)', 'Filmtrust', 'The Image...",NaN
4448,tt0025854,tt0025854,movie,Man's Way with Women,Sången om den eldröda blomman,0.000,1934.000,NaN,96,NaN,Le chant de la fleur rouge,5.000,9.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2415,tt0016654,tt0016654,movie,The Black Pirate,The Black Pirate,0.000,1926.000,NaN,88,"Action,Adventure",Le pirate noir,7.000,2128.000,False,/ezeFFebkb8MX33IRQ2LYwegLNit.jpg,1300000.000,"['Adventure', 'Drama', 'Action']",NaN,50075.000,tt0016654,en,The Black Pirate,A nobleman vows to avenge the death of his fat...,3.959,/841WLSvTRN3TQKxiUjD1TY9dc42.jpg,['US'],1926-03-08,0.000,94.000,['xx'],Released,The Love Story of a Bold Buccaneer,The Black Pirate,False,7.000,43.000,"['United Artists', 'Elton Corporation']",NaN
56239,tt1385478,tt1385478,movie,As We Forgive,As We Forgive,0.000,2009.000,NaN,53,Documentary,As We Forgive,8.100,58.000,False,NaN,0.000,['Documentary'],http://www.asweforgivemovie.com/,50306.000,tt1385478,en,As We Forgive,Could you forgive a person who murdered your f...,0.772,/pWKYa9GBK4CLAU3oAEdvYQxiHiZ.jpg,[],2008-01-01,0.000,53.000,[],Released,Could you forgive a person who murdered your f...,As We Forgive,False,6.000,3.000,[],NaN
73543,tt4162100,tt4162100,movie,Antoine & Marie,Antoine et Marie,0.000,2014.000,NaN,85,Drama,Antoine et Marie,5.600,79.000,False,/5xjW0kRbon7tfJ7U0SWcQVuMhKO.jpg,0.000,['Drama'],NaN,314644.000,tt4162100,fr,Antoine et Marie,Marie leads a calm and happy life withe her bo...,0.846,/qMdAKDup3vNiaaE6wUUBRjlRPYI.jpg,['CA'],2014-10-16,0.000,85.000,['fr'],Released,NaN,Antoine et Marie,False,5.000,5.000,['DPOST'],NaN
155,tt0006488,tt0006488,movie,Caprice of the Mountains,Caprice of the Mountains,0.000,1916.000,NaN,75,Drama,La fleur enchantée,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81601,tt8366590,tt8366590,movie,Baaghi 3,Baaghi 3,0.000,2020.000,NaN,143,"Action,Adventure,Thriller",Baaghi 3,2.200,13831.000,False,/nQV1lkgGn0FhqDeDr4b8VLhn1ow.jpg,7952450.000,"['Action', 'Adventure', 'Thriller']",NaN,594669.000,tt8366590,hi,बाघी ३,A man embarks on a bloody rampage to save his ...,8.816,/14nxzkz2ZE0BXoywF6UnP5rjfcL.jpg,['IN'],2020-03-06,0.000,143.000,"['hi', 'tr']",Released,This time He's up against a Nation,Baaghi 3,False,7.130,92.000,"['Nadiadwala Grandson Entertainment', 'Star St...","['IN', 'IN']"
1600,tt0013679,tt0013679,movie,Tillie,Tillie,0.000,1922.000,NaN,50,Drama,Le roman de cousine Laure,7.200,18.000,False,NaN,0.000,['Drama'],NaN,751110.000,tt0013679,en,Tillie,"Tillie Getz (Minter), the eldest daughter of J...",0.600,/7oJzlG76vHOAXPJTYnjeILviLYz.jpg,[],1922-01-30,0.000,50.000,[],Released,NaN,Tillie,False,0.000,0.000,"['Realart Pictures Corporation', 'Paramount']","['US', 'US']"
64633,tt21816560,tt21816560,movie,A Cooler Climate,A Cooler Climate,0.000,2022.000,NaN,75,Documentary,Un été afghan,6.100,57.000,False,/zEJZPTJLjoCKQZwoxdSI2nEDLi7.jpg,0.000,['Documentary'],NaN,1013569.000,tt21816560,en,A Cooler Climate,Award-winning filmmaker James Ivory recounts h...,1.022,/uiXeQmqjCgXRMJpSInmLEuwWWGd.jpg,['US'],2022-10-07,0.000,75.000,[],Releas

In [38]:
# Which TMdb genre for NA genre in IMDB
movie_final[movie_final["genres_x"].isna()]["genres_y"].unique()

array(['[]', nan, "['Drama', 'History']", "['Drama']",
       "['Science Fiction']", "['Mystery', 'Crime']", "['Comedy']",
       "['Adventure']", "['Drama', 'Comedy']", "['Crime', 'Drama']",
       "['Comedy', 'Drama']", "['Drama', 'Action', 'Adventure']",
       "['Music', 'Comedy']", "['Adventure', 'Drama']",
       "['Romance', 'Comedy']", "['Romance', 'Drama']",
       "['Drama', 'War']", "['Comedy', 'Music']", "['Drama', 'Music']",
       "['Music', 'Romance', 'Drama']", "['Family', 'Music']",
       "['Music']", "['Crime']", "['Romance']", "['Drama', 'Family']",
       "['Documentary']", "['Comedy', 'Drama', 'Romance']",
       "['Drama', 'Documentary']", "['Thriller']", "['Crime', 'Comedy']",
       "['Thriller', 'Drama']", "['Family', 'Drama']",
       "['Comedy', 'Romance']", "['Drama', 'Romance']",
       "['Action', 'Comedy']", "['Action', 'Adventure', 'History']",
       "['History']", "['Crime', 'Action', 'Drama']",
       "['Crime', 'Action']", "['Drama', 'Drama']",
    

In [39]:
to_add = movie_final[(movie_final["genres_x"].isna()) & (movie_final["genres_y"].notna()) & (movie_final["genres_y"] != "[]")]

In [41]:
movie_final.drop(movie_final[movie_final["genres_x"].isna()].index, inplace = True)

In [43]:
movie_final = pd.concat([movie_final, to_add])

In [46]:
movie_final["status"].value_counts()

Released           59683
Post Production      326
In Production        217
Planned               92
Canceled               3
Rumored                2
Name: status, dtype: int64

In [47]:
movie_final = movie_final[movie_final["status"] == "Released"]

In [49]:
movie_final.drop(columns = ["key_0", "titleType", "endYear", "adult", "budget", "homepage", "production_countries", "original_title", "revenue", "spoken_languages", "status", "tagline", "video"], inplace = True)

In [51]:
movie_final["release_date"].fillna(movie_final["startYear"], inplace = True)

In [52]:
movie_final.dropna(subset = "release_date", inplace = True)

In [53]:
movie_final["release_date"] = movie_final["release_date"].apply(lambda x: str(x)[:4])

In [54]:
movie_final["averageRating"].fillna(movie_final["vote_average"], inplace = True)

In [55]:
movie_final["runtimeMinutes"].fillna(movie_final["runtime"], inplace = True)

In [56]:
movie_final.dropna(subset = "runtimeMinutes", inplace = True)

In [57]:
movie_final["runtimeMinutes"] = movie_final["runtimeMinutes"].apply(lambda x: int(x))

In [58]:
movie_final["numVotes"].fillna(movie_final["vote_count"], inplace = True)

In [59]:
movie_final.drop(columns = ["startYear", "runtime", "vote_average", "vote_count"], inplace = True)

In [60]:
movie_final["genres_x"].fillna(movie_final["genres_y"], inplace = True)

movie_final["genres_x"].isna().value_counts()

False    59682
Name: genres_x, dtype: int64

In [63]:
movie_final["genres_x"] = movie_final["genres_x"].apply(lambda x: x.replace("[", "").replace("]", "").replace("'", "").replace(" ", ""))

In [64]:
movie_final["genres_x"] = movie_final["genres_x"].apply(lambda x: x.split(","))

In [65]:
#movie_final.to_csv('movie_final.csv')

In [66]:
movie_final["genres_x"].explode("genres_x").unique()

array(['Drama', 'Adventure', 'Fantasy', 'Biography', 'Romance', 'History',
       'Comedy', 'Crime', 'Mystery', 'Horror', 'Western', 'Action',
       'Family', 'War', 'Sci-Fi', 'Thriller', 'Sport', 'Documentary',
       'Music', 'Animation', 'Musical', 'Film-Noir', 'News',
       'ScienceFiction'], dtype=object)

In [67]:
def repl_list(listing, old_word, new_word):
    for ind, ele in enumerate(listing):
        if ele == old_word:
            listing[ind] = new_word
    return listing

In [68]:
movie_final["genres_x"] = movie_final["genres_x"].apply(lambda x: repl_list(x, 'ScienceFiction', 'Sci-Fi'))

In [69]:
movie_final["genres_x"] = movie_final["genres_x"].apply(lambda x: repl_list(x, 'Musical', 'Music'))


In [71]:
len(movie_final["genres_x"].explode().unique())

22

In [72]:
test =pd.DataFrame( movie_final['averageRating'].sort_values(ascending = False))
test['genre'] = movie_final['genres_x']
test['title'] = movie_final['title_x']


,averageRating,genre,title
83719,10.000,"[Biography, Documentary, Family]",Je ne sais pas quelle ville est à ses yeux
33867,10.000,[Comedy],Mon ami Victor
45966,10.000,[Documentary],L'esprit de la route
8083,10.000,[Drama],Les noces de sable
58951,10.000,[Fantasy],Purplemind
...,...,...,...
47042,0.000,[Documentary],Liberté
62881,0.000,"[Drama, Fantasy]",Nymphose
18522,0.000,"[Action, Crime, Drama]",Karaté sanglant
79032,0.000,[Documentary],Tes cheveux démêlés cachent une guerre de 7 ans


In [73]:
test.reset_index()

,index,averageRating,genre,title
0,83719,10.000,"[Biography, Documentary, Family]",Je ne sais pas quelle ville est à ses yeux
1,33867,10.000,[Comedy],Mon ami Victor
2,45966,10.000,[Documentary],L'esprit de la route
3,8083,10.000,[Drama],Les noces de sable
4,58951,10.000,[Fantasy],Purplemind
...,...,...,...,...
59677,47042,0.000,[Documentary],Liberté
59678,62881,0.000,"[Drama, Fantasy]",Nymphose
59679,18522,0.000,"[Action, Crime, Drama]",Karaté sanglant
59680,79032,0.000,[Documentary],Tes cheveux démêlés cachent une guerre de 7 ans


We create as much columns as genre to "get_dummie" the variable one by one to consider the multi genre films

In [74]:
genre_list = movie_final["genres_x"].explode().unique()

for genr in genre_list:
    movie_final[genr] = movie_final["genres_x"].apply(lambda x: 1 if genr in (x) else 0)

In [75]:
movie_final.sample(10)

,tconst,primaryTitle,originalTitle,isAdult,runtimeMinutes,genres_x,title_x,averageRating,numVotes,backdrop_path,genres_y,id,imdb_id,original_language,overview,popularity,poster_path,release_date,title_y,production_companies_name,production_companies_country,Drama,Adventure,Fantasy,Biography,Romance,History,Comedy,Crime,Mystery,Horror,Western,Action,Family,War,Sci-Fi,Thriller,Sport,Documentary,Music,Animation,Film-Noir,News
54756,tt12989912,Ultrasound,Ultrasound,0.000,103,"[Drama, Mystery, Sci-Fi]",Ultrasound,5.800,1677.000,/o6UwCnhoezCWyTbZE76zipyjHKc.jpg,['Science Fiction'],821510.000,tt12989912,en,"After his car breaks down, Glen spends one hel...",5.646,/xef7fVF5vhDPQk7yQ0dJwKHIev2.jpg,2022,Ultrasound,['Lodger Films'],[''],1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
38391,tt0218641,Submerged,Submerged,0.000,95,[Action],Péril en haute mer,3.300,834.000,/1ePgTmNASS1kH6VuC5Q7FfexFgD.jpg,['Action'],79949.000,tt0218641,en,Navy SEALs launch a rescue mission after terro...,4.029,/7t5CDAFWpjYh4cFyjIO3d3wDAYK.jpg,2000,Submerged,[],NaN,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
14866,tt0059116,The Possessed,La donna del lago,0.000,95,"[Crime, Drama, Horror]",La femme du lac,7.000,1420.000,/mzNAJqEjFevV3qr746yEwljT2Sy.jpg,"['Mystery', 'Crime', 'Drama', 'Thriller']",104244.000,tt0059116,it,While visiting his favorite resort village dur...,2.849,/cwa9oWlNqUNrDCuwir3y84iKK8P.jpg,1965,The Possessed,"['Istituto Luce Cinecittà', 'B.R.C. Produzione...",NaN,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9115,tt0043813,The Model and the Marriage Broker,The Model and the Marriage Broker,0.000,103,"[Comedy, Romance]",Agence cupidon,7.000,892.000,/8kg5nSI3px7QEOHMpDwm0yi39xx.jpg,"['Drama', 'Comedy']",132094.000,tt0043813,en,A marriage broker can't resist meddling in the...,0.840,/bL00Xs7GYV1ofGJk4YPWqusNWMz.jpg,1951,The Model and the Marriage Broker,['20th Century Fox'],NaN,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23648,tt0088763,Back to the Future,Back to the Future,0.000,116,"[Adventure, Comedy, Sci-Fi]",Retour vers le futur,8.500,1303700.000,/5bzPWQ2dFUl2aZKkp7ILJVVkRed.jpg,"['Adventure', 'Comedy', 'Science Fiction']",105.000,tt0088763,en,Eighties teenager Marty McFly is accidentally ...,53.454,/fNOH9f1aA7XRTzl1sAOx9iF553Q.jpg,1985,Back to the Future,"['Universal Pictures', 'Amblin Entertainment']","['US', 'US']",0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
21780,tt0081197,Bordello,Las mujeres de Jeremías,0.000,98,"[Adventure, Comedy, Drama]",Las mujeres de Jeremías,5.800,74.000,NaN,"['Western', 'Adventure', 'Comedy']",107621.000,tt0081197,es,Minister with four virgin daughters inherits a...,1.638,/nLSP3BIpAczl8fiwJ8HmymiGNjj.jpg,1981,Bordello,"['Ízaro Films', 'Producciones Esme S.A.']",NaN,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
44582,tt0405096,Ladybirds' Christmas,Lepatriinude jõulud,0.000,54,"[Animation, Family]",Le Noël des coccinelles,7.200,255.000,/yqb2fqrmyOPh3FftSUM4vfBzwiV.jpg,"['Family', 'Animation']",182121.000,tt0405096,et,Two young ladybirds Tim and Mia are unexpected...,0.742,/brpoU0pjnRWNGlQ4rklgv7fvBrO.jpg,2001,Ladybirds' Christmas,['Eesti Joonisfilm'],NaN,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
35640,tt0188506,A Monkey's Tale,Le château des singes,0.000,76,"[Adventure, Animation, Family]",Le château des singes,5.700,527.000,/oG24VU7BdJnw8YeXYvp0bcQzzjU.jpg,"['Adventure', 'Animation', 'Family']",37949.000,tt0188506,fr,Kom is from a tribe of monkeys who live in a c...,5.430,/s4pYSLnGHXUyiTs0HNmpUJMYVKv.jpg,1999,A Monkey's Tale,"['Steve Walsh Productions', 'Cologne Cartoon',...",NaN,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
40807,tt0282628,Jan Dara,Jan Dara,0.000,108,"[Drama, Romance]",Jan Dara,6.000,2433.000,/vvEt03Fzgc4NujpknFfR1LrXgno.jpg,"['Romance', 'Drama']",68507.000,tt0282628,th,Jan is a boy growing up in 1930s Siam in a wea...,21.891,/yPSBDsLZhA3BkKSE4LrPxoIv5mE.jpg,2001,Jan Dara,['Tai Entertainment'],NaN,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12780,tt0053724,El cochecito,El cochecito,0.000,85,"[Comedy, Dr

In [76]:
movie_final[['originalTitle', 'primaryTitle', 'title_y', 'title_x']].sample(5)

,originalTitle,primaryTitle,title_y,title_x
36040,L'esclave blanche,Pasha's Wives,Pasha's Wives,L'esclave blanche
13676,Gypsy,Gypsy,Gypsy,"Gypsy, vénus de Broadway"
59768,Savages,Savages,Savages,Savages
81704,15+ Coming of Age,15+ Coming of Age,15+ Coming of Age,15+ Coming of Age
14148,Kings of the Sun,Kings of the Sun,Kings of the Sun,Les rois du soleil
35645,Island of Death,Island of Death,Island of Death,L'île des morts
5133,Easy Living,Easy Living,Easy Living,Vie facile
63573,Pursuit of Loneliness,Pursuit of Loneliness,Pursuit of Loneliness,Pursuit of Loneliness
27470,Fei hu wai zhuan,The Sword of Many Lovers,The Sword of Many Lovers,Le poison et l'épée
62855,Madangeul naon amtak,"Daisy, a Hen Into the Wild","Leafie, a Hen Into the Wild",Lili à la découverte du monde sauvage


In [77]:
movie_final.drop(columns = ["primaryTitle", "title_y", "genres_y"], inplace = True)

In [79]:
movie_final.drop(columns = ["production_companies_name", "production_companies_country"], inplace = True)

In [80]:
list_of_movie = list(movie_final['tconst'].unique())

In [81]:
title_prin1 = title_prin[title_prin["tconst"].isin(list_of_movie)]

In [83]:
writers_only = title_prin1[title_prin1['category']=='writer']
df2 = writers_only.sort_values(by=['tconst', 'ordering'])
df2['ordering'] = df2.groupby('tconst').cumcount() + 1
pivot_writers = df2.pivot(index='tconst', columns='ordering', values='nconst')
pivot_writers.columns = [f'writer{i}' for i in pivot_writers.columns]
pivot_writers = pivot_writers.reset_index()
pivot_writers.drop(columns = ['writer2', 'writer3', 'writer4'], inplace = True)
pivot_writers

,tconst,writer1
0,tt0000591,nm0141150
1,tt0001790,nm0135053
2,tt0002130,nm0019604
3,tt0002199,nm0310155
4,tt0002375,nm0304098
...,...,...
57683,tt9907782,nm1193346
57684,tt9908390,nm1597648
57685,tt9911196,nm0277932
57686,tt9913084,nm0812850


In [92]:
directors_only = title_prin1[title_prin1['category']=='director']

In [93]:
df1 = directors_only.sort_values(by=['tconst', 'ordering'])

In [94]:
df1['ordering'] = df1.groupby('tconst').cumcount() + 1
pivot_directors = df1.pivot(index='tconst', columns='ordering', values='nconst')

In [95]:
pivot_directors.columns = [f'director{i}' for i in pivot_directors.columns]
pivot_directors = pivot_directors.reset_index()

In [96]:
pivot_directors.drop(columns=['director2', 'director3', 'director4'], inplace = True)

In [97]:
pivot_directors

,tconst,director1
0,tt0000591,nm0141150
1,tt0001790,nm0135052
2,tt0002130,nm0078205
3,tt0002199,nm0646058
4,tt0002375,nm0135052
...,...,...
59430,tt9908390,nm1415268
59431,tt9911196,nm0631590
59432,tt9913084,nm0812850
59433,tt9915790,nm10538030


In [98]:
crew = pd.merge(pivot_directors, pivot_writers, how = 'left', on='tconst')
crew

,tconst,director1,writer1
0,tt0000591,nm0141150,nm0141150
1,tt0001790,nm0135052,nm0135053
2,tt0002130,nm0078205,nm0019604
3,tt0002199,nm0646058,nm0310155
4,tt0002375,nm0135052,nm0304098
...,...,...,...
59430,tt9908390,nm1415268,nm1597648
59431,tt9911196,nm0631590,nm0277932
59432,tt9913084,nm0812850,nm0812850
59433,tt9915790,nm10538030,NaN


In [99]:
actors_only = title_prin1[(title_prin1['category']=='actor') & (title_prin1['ordering'] <=4)]

In [100]:
df = actors_only.sort_values(by=['tconst', 'ordering'])

In [101]:
df['ordering'] = df.groupby('tconst').cumcount() + 1

In [102]:
pivot_df = df.pivot(index='tconst', columns='ordering', values='nconst')

In [103]:
pivot_df.columns = [f'actor{i}' for i in pivot_df.columns]
pivot_df = pivot_df.reset_index()

In [104]:
pivot_df.drop(columns = ['actor4'], inplace = True)

In [106]:
casts_only = pd.merge(crew, pivot_df,  how='left', on='tconst')

In [131]:
movie_f = pd.merge(movie_final,casts_only, how = 'left', left_on = 'tconst', right_on = 'tconst')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 59682 entries, 0 to 59681
Data columns (total 43 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             59682 non-null  object 
 1   originalTitle      59682 non-null  object 
 2   isAdult            59682 non-null  float64
 3   runtimeMinutes     59682 non-null  int64  
 4   genres_x           59682 non-null  object 
 5   title_x            59682 non-null  object 
 6   averageRating      59682 non-null  float64
 7   numVotes           59682 non-null  float64
 8   backdrop_path      47754 non-null  object 
 9   id                 59682 non-null  float64
 10  imdb_id            59682 non-null  object 
 11  original_language  59682 non-null  object 
 12  overview           57908 non-null  object 
 13  popularity         59682 non-null  float64
 14  poster_path        58112 non-null  object 
 15  release_date       59682 non-null  object 
 16  Drama              596

In [133]:
movie_f['isAdult'] = movie_f['isAdult'].apply(lambda x : int(x))
movie_f['numVotes'] = movie_f['numVotes'].apply(lambda x : int(x))
movie_f['id'] = movie_f['id'].apply(lambda x : int(x))

In [134]:
import numpy as np
def non_dispo(x):
    if x is np.nan:
        return "non renseigné"
    else : 
        return x 

In [135]:
movie_f['director1'] = movie_f['director1'].apply(lambda x: non_dispo(x))
movie_f['writer1'] = movie_f['writer1'].apply(lambda x: non_dispo(x))
movie_f['actor1'] = movie_f['actor1'].apply(lambda x: non_dispo(x))
movie_f['actor2'] = movie_f['actor2'].apply(lambda x: non_dispo(x))
movie_f['actor3'] = movie_f['actor3'].apply(lambda x: non_dispo(x))


In [136]:
movie_f["genres_x"] =movie_f["genres_x"].apply(lambda x: ' '.join (str(ele) for ele in x))


In [139]:
movie_f['original_language']=movie_f['original_language'].apply(lambda x: x.upper() )

In [141]:
movie_f.to_csv('movie_final_ML1.csv')

In [236]:
movie_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59682 entries, 0 to 59681
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tconst             59682 non-null  object 
 1   originalTitle      59682 non-null  object 
 2   isAdult            59682 non-null  int64  
 3   runtimeMinutes     59682 non-null  int64  
 4   genres_x           59682 non-null  object 
 5   title_x            59682 non-null  object 
 6   averageRating      59682 non-null  float64
 7   numVotes           59682 non-null  float64
 8   backdrop_path      47754 non-null  object 
 9   id                 59682 non-null  float64
 10  imdb_id            59682 non-null  object 
 11  original_language  59682 non-null  int64  
 12  overview           57908 non-null  object 
 13  popularity         59682 non-null  float64
 14  poster_path        58112 non-null  object 
 15  release_date       59682 non-null  object 
 16  Drama              596

# THE END IS HERE 
